In [0]:
import math
import io
import os
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import tree

from tensorflow.examples.tutorials.mnist import input_data


In [1]:
!apt-get install graphviz
!pip install graphviz
import graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig libann0 libcairo2 libcdt5 libcgraph6 libdatrie1 libgd3
  libgts-0.7-5 libgts-bin libgvc6 libgvpr2 libjbig0 liblab-gamut1 libltdl7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4
  libpixman-1-0 libthai-data libthai0 libtiff5 libwebp6 libxaw7 libxcb-render0
  libxcb-shm0 libxmu6 libxpm4 libxt6
Suggested packages:
  gsfonts graphviz-doc libgd-tools
The following NEW packages will be installed:
  fontconfig graphviz libann0 libcairo2 libcdt5 libcgraph6 libdatrie1 libgd3
  libgts-0.7-5 libgts-bin libgvc6 libgvpr2 libjbig0 liblab-gamut1 libltdl7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4
  libpixman-1-0 libthai-data libthai0 libtiff5 libwebp6 libxaw7 libxcb-render0
  libxcb-shm0 libxmu6 libxpm4 libxt6
0 upgraded, 30 newly installed, 0 to remove and 4 not upgraded.
Need to get 4,154 kB of

## Training an MNIST network (2 conv layers + 1 fully connected)

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

tf.reset_default_graph()
sess = tf.InteractiveSession()

BATCH_SIZE = 50
TEST_BATCH_SIZE = 1000  
IMAGE_SIZE = 28
NUM_CLASSES = 10

# t_raw_training_images, t_training_labels = tf.LegacyImageInput(TRAINING_DATA, BATCH_SIZE, IMAGE_SIZE)
# t_raw_test_images, t_test_labels = tf.LegacyImageInput(EVAL_DATA, TEST_BATCH_SIZE, IMAGE_SIZE)
# t_training_images = tf.reshape(t_raw_training_images, [BATCH_SIZE, IMAGE_SIZE ** 2])
# t_test_images = tf.reshape(t_raw_test_images, [TEST_BATCH_SIZE, IMAGE_SIZE ** 2])

# t_training_labels = tf.nn.label_classes(t_training_labels, NUM_CLASSES)
#t_test_labels = tf.nn.label_classes(t_test_labels, NUM_CLASSES)


# test_images, test_labels = sess.run([t_test_images, t_test_labels])

x = tf.placeholder(tf.float32, shape=[None, 784], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name='thought')

keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.add(tf.matmul(h_fc1_drop, W_fc2), b_fc2, name='prediction')

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)

sess.run(tf.global_variables_initializer())

In [6]:
mnist = input_data.read_data_sets('MNIST_data')

for i in range(0, 800):
  batch = mnist.train.next_batch(BATCH_SIZE)
  # batch = sess.run([t_training_images, t_training_labels])
  train_step.run(feed_dict={x: batch[0], y_: np.eye(10)[batch[1]], keep_prob: 0.5})
  if i%100 == 0:
    test_accuracy = accuracy.eval(feed_dict={
        x:mnist.test.images, y_: np.eye(10)[mnist.test.labels], keep_prob: 1.0})
    print("step %d, test accuracy %g"%(i, test_accuracy))
  #if i%500 == 0:
    #saver.save(sess,'/usr/local/google/home/ataly/Research/Models/MNIST/model.ckpt', global_step=i)
    #print "Model saved after %d steps" % i

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
step 0, test accuracy 0.0946
step 100, 

## Library for IG Attribution and Conductance

In [0]:
t_label = tf.placeholder(tf.int32)
t_neuron_id = tf.placeholder(tf.int32)
t_grad = tf.gradients(y_conv[:, t_label], x)
t_grad_neuron = tf.gradients(y_conv[:, t_label], h_fc1)[0]
t_grad_conductance = tf.gradients(h_fc1[:,t_neuron_id], x, grad_ys=t_grad_neuron[:, t_neuron_id])

In [0]:
import copy
layer_param_tensors = [
    [W_conv1, b_conv1],
    [W_conv2, b_conv2],
    [W_fc1, b_fc1],
    [W_fc2, b_fc2]
]
  
def get_prediction(inps, tensor=y_conv, batch_size=100):
  def get_prediction_batch(batch):
    feed = {x: batch, keep_prob:1.0}
    return sess.run(tensor, feed_dict=feed)
  n = len(inps)
  if n%batch_size == 0:
    batches = [inps[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size))]
  else:
    batches = [inps[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size) +1)]    
  batch_predictions = [get_prediction_batch(b) for b in batches]
  return np.concatenate(tuple(batch_predictions), axis=0)

def attribute(inp, label, baseline=None, steps=50, use_top_label=False):
  def top_label(inp):
    return np.argmax(get_prediction([inp])[0])
  if baseline is None:
    baseline = 0*inp
  scaled_inputs = [baseline + (float(i)/steps)*(inp-baseline) for i in range(0, steps)]
  feed = {keep_prob:1.0}
  if use_top_label:
    feed[x] = [inp]
    logits = sess.run(y_conv, feed_dict=feed)[0]
    label = np.argmax(logits)
  feed[x] = scaled_inputs
  feed[t_label] = label
  grads, scores = sess.run([t_grad, y_conv], feed_dict=feed)  # shapes: <steps+1>, <steps+1, inp.shape>
  integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
  print "FINAL SCORE", scores[-1][label]
  print "BASELINE SCORE", scores[0][label]
  print "SUM", np.sum(integrated_gradients), "DIFF", scores[-1][label] - scores[0][label]
  return integrated_gradients

def conductance(inp, label, neuron_id=None, baseline=None, steps=50):
  # neuron_id is the id of the neuron in layer h_fc1 through which conductance
  # must be computed. If None, vanilla IG is computed.
  if baseline is None:
    baseline = 0*inp
  scaled_inputs = [baseline + (float(i)/steps)*(inp-baseline) for i in range(0, steps)]
  feed = {keep_prob:1.0}
  feed[x] = scaled_inputs
  feed[t_label] = label
  if neuron_id != None:
    feed[t_neuron_id] = neuron_id
    grads, scores = sess.run([t_grad_conductance, y_conv], feed_dict=feed)  # shapes: <steps+1>, <steps+1, inp.shape>
    integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
    return integrated_gradients
  grads, scores = sess.run([t_grad, y_conv], feed_dict=feed)  # shapes: <steps+1>, <steps+1, inp.shape>    
  integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
  print "FINAL SCORE", scores[-1][label]
  print "BASELINE SCORE", scores[0][label]
  print "SUM", np.sum(integrated_gradients), "DIFF", scores[-1][label] - scores[0][label]
  return integrated_gradients

## Library for Visualizing Images and Attributions

In [0]:
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
import numpy as np
from cStringIO import StringIO

In [0]:
FONT_PATH='/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed.ttf'
IMAGE_SIZE = 28

def mnist_to_rgb(mnist_img):
  """
  Transformsn an MNIST image (shape: <784>) to a grayscale
  RGB image (shape: <28,28,3>)
  """
  pixel_array = mnist_img.reshape(IMAGE_SIZE, IMAGE_SIZE)  # shape: 28,28
  rgb_image = np.transpose([pixel_array,pixel_array,pixel_array], axes=[1,2,0])
  return rgb_image

def pil_img(a):
  '''Returns a PIL image created from the provided RGB array.
  '''
  a = np.uint8(a)
  return PIL.Image.fromarray(a)

def mnist_to_pil_img(inp):
  rgb_inp = 255*mnist_to_rgb(inp)
  vis_inp = pil_img(rgb_inp)
  return vis_inp  

def pil_fig(fig):
  # Returns a PIL image obtained from the provided PLT figure.
  buf = io.BytesIO()
  fig.savefig(buf, format='png')
  plt.close(fig)
  buf.seek(0)
  img = PIL.Image.open(buf)
  return img

def show_img(img, fmt='jpeg'):
  '''Displays the provided PIL image
  '''
  f = StringIO()
  img.save(f, fmt)
  display(Image(data=f.getvalue()))
 
def show_mnist_img(mnist_img):
  show_img(pil_img(255*mnist_to_rgb(mnist_img)))
  
def gray_scale(img):
  '''Converts the provided RGB image to gray scale.
  '''
  img = np.average(img, axis=2)
  return np.transpose([img, img, img], axes=[1,2,0])

def normalize(attrs, ptile=99):
  '''Normalize the provided attributions so that they fall between
     -1.0 and 1.0.
  '''
  h = np.percentile(attrs, ptile)
  l = np.percentile(attrs, 100-ptile)
  return np.clip(attrs/max(abs(h), abs(l)), -1.0, 1.0)    

def pil_text(strs, shape, start_h=10, start_w=10, font_size=18, color=(0, 0, 0)):
  # Returns a PIL image with the provided text.
  img = pil_img(255*np.ones(shape))
  draw = PIL.ImageDraw.Draw(img)
  font = PIL.ImageFont.truetype(FONT_PATH, font_size)
  h = start_h
  for s in strs: 
    draw.text((start_w,h), s, fill=color, font=font)
    h = h + 30
  return img

def combine(imgs, horizontal=True):
  # Combines the provided PIL Images horizontally or veritically
  if horizontal:
    w = np.sum([img.size[0]+10 for img in imgs])
    h = np.max([img.size[1] for img in imgs])
  else:
    w = np.max([img.size[0] for img in imgs])
    h = np.sum([img.size[1]+10 for img in imgs])
  final_img = PIL.Image.new('RGB', (w, h), color='white')
  pos = 0
  for img in imgs:
    if horizontal:
      final_img.paste(im=img, box=(pos,0))
      pos = pos+img.size[0]+10
    else:
      final_img.paste(im=img, box=(0,pos))
      pos = pos+img.size[1]+10
  return final_img

def visualize_attrs(img, attrs, ptile=99):
  '''Visaualizes the provided attributions by first aggregating them
    along the color channel to obtain per-pixel attributions and then
    scaling the intensities of the pixels in the original image in
    proportion to absolute value of these attributions.

    The provided image and attributions must of shape (224, 224, 3).
  '''
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
  attrs = gray_scale(attrs)
  attrs = abs(attrs)
  attrs = np.clip(attrs/np.percentile(attrs, ptile), 0,1)
  vis = img*attrs
  return pil_img(vis)
  
  
R=np.array([255,0,0])
G=np.array([0,255,0])
B=np.array([0,0,255])
def visualize_attrs2(img, attrs, pos_ch=G, neg_ch=R, ptile=99):
  '''Visaualizes the provided attributions by first aggregating them
     along the color channel and then overlaying the positive attributions
     along pos_ch, and negative attributions along neg_ch.

     The provided image and attributions must of shape (224, 224, 3).
  '''
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
  attrs = gray_scale(attrs)
  attrs = normalize(attrs, ptile)   
  pos_attrs = attrs * (attrs >= 0.0)
  neg_attrs = -1.0 * attrs * (attrs < 0.0)
  attrs_mask = pos_attrs*pos_ch + neg_attrs*neg_ch
  vis = 0.3*gray_scale(img) + 0.7*attrs_mask
  return pil_img(vis)

## Extracting Invariant Candidates

In [0]:
def fingerprint_suffix(inps):
  # Below h_fc1 is the final fully connected layer of size 1024.
  return (get_prediction(inps, tensor=h_fc1)>0.0).astype('int')

def fingerprint_prefix(inps):
  return (get_prediction(inps, tensor=tf.reshape(h_pool1, [-1, 14*14*32]))>0.0).astype('int')

def describe_input(i):
  print "Groundtruth", mnist.train.labels[i]
  print "Prediction", all_predictions[i]
  show_mnist_img(mnist.train.images[i])

In [0]:
# all_suffixes, all_prefixes, all_predictions are in the same order
# as mnist.train.images. Henceforth when we use the index i we will
# be referring to mnist.train.images[i].
all_suffixes = fingerprint_suffix(mnist.train.images)
all_prefixes = fingerprint_prefix(mnist.train.images)
all_predictions = np.argmax(get_prediction(mnist.train.images), axis=1)

### Build the Decision Tree

In [13]:
estimator = tree.DecisionTreeClassifier()
estimator.fit(all_suffixes, all_predictions)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Examine clusters/invariants corresponding to different inputs

In [0]:
def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(i, neuron_ids=range(1024)):
  # Get the cluster of inputs that contains input at index i
  # and such that all inputs in the cluster have the same
  # signature for the provided neurons.
  #
  # When neuron_ids is set to all indices in range(1024),
  # we are effectively asking for the cluster where all inputs
  # have the same suffix as that of input at index i. 
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  ref_suffix = fingerprint_suffix([mnist.train.images[i]])
  ref_neuron_sig = ref_suffix[0, neuron_ids]
  cluster = []
  for i, s in enumerate(all_suffixes):
    sig = s[neuron_ids]
    if np.array_equal(ref_neuron_sig, sig):
      cluster.append(i)
  return np.array(cluster)

def is_consistent_cluster(cluster):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = all_predictions[cluster[0]]
  for i in cluster:
    if all_predictions[i] != pred:
      return False
  return True

def is_misclassified(i):
  return all_predictions[i] != mnist.train.labels[i]

def visualize_conductances(img, label, neuron_ids, only_on=False):
  # Visualize the conductances for the provided image.
  # Args:
  # - img: the provided mnist image
  # - label: prediction label w.r.t. conductance must be computed
  # - neuron_ids: list of neurons indices from the suffix tensor for which
  #    conductances must be computed.
  # - only_on: If True then conductance is computed only for those neurons
  #    that are on for the given image. 
  vis = [mnist_to_pil_img(img)]
  suffix = fingerprint_suffix([img])
  for i, id in enumerate(neuron_ids):
    if only_on and suffix[i] != 1:
      continue  
    igc = conductance(img, label, neuron_id=id)
    # igc = conductances[id]
    vis.append(visualize_attrs2(255*mnist_to_rgb(img), mnist_to_rgb(igc)))
  return combine(vis)

def gen_invariant(ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_prediction = all_predictions[ref_id]
  ref_suffix = all_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  cluster = get_suffix_cluster(ref_id, neuron_ids)
  return cluster, neuron_ids

def describe_cluster(cluster, neuron_ids):
  neuron_sig = all_suffixes[cluster[0]][neuron_ids]
  print "Num neurons in invariant", len(neuron_ids)
  print "Neuron id and signature", zip(neuron_ids, neuron_sig)
  print "Cluster size: ", len(cluster)
  print "Is Consistent cluster: ", is_consistent_cluster(cluster)
  print "Num misclassified", len([i for i in cluster if is_misclassified(i)])


In [17]:
# ref_id is the index of the reference input
ref_id = 1
print "### Reference Image ###"
describe_input(ref_id)
print "### Cluster ###"
cluster, neuron_ids = gen_invariant(ref_id)
describe_cluster(cluster, neuron_ids)

# Visualize conductances for 10 inputs in the cluster
for i in cluster[:10]:
  show_img(visualize_conductances(mnist.train.images[i], all_predictions[i], neuron_ids, only_on=False))

### Reference Image ###
Groundtruth 8
Prediction 8


### Cluster ###
Num neurons in invariant 15
Neuron id and signature [(129, 0), (201, 1), (14, 0), (410, 1), (507, 0), (480, 1), (326, 0), (80, 0), (749, 0), (288, 1), (938, 0), (792, 1), (412, 0), (441, 1), (753, 0)]
Cluster size:  91
Is Consistent cluster:  True
Num misclassified 1


### Analysing clusters of misclassified inputs

In [0]:
# mis_six contains all the misclassified sixes in the training set.
mis_six = []
for i in range(len(mnist.train.images)):
  if all_predictions[i] == mnist.train.labels[i]:
    continue
  if mnist.train.labels[i] == 6:
    mis_six.append(i)

In [0]:
# Let us analyze the cluster for 10 of these misclassified sixes.
for i in mis_six[:10]:
  print "### Reference Image ###"
  describe_input(i)
  print "### Cluster ###"
  cluster, neuron_ids, neuron_sig = gen_invariant(i)
  describe_cluster(cluster)
  print ""
  

### Reference Image ###
Groundtruth 6
Prediction 0


### Cluster ###
Num neurons in invariant 23
Neurion id and signature [(447, 0), (936, 1), (51, 1), (854, 0), (240, 1), (948, 0), (87, 1), (345, 0), (816, 1), (908, 0), (98, 0), (730, 0), (332, 0), (420, 1), (667, 0), (40, 0), (440, 0), (923, 1), (813, 0), (859, 0), (294, 0), (786, 0), (844, 0)]
Cluster size:  8
Is Consistent cluster:  True
Num misclassified 1

### Reference Image ###
Groundtruth 6
Prediction 4


### Cluster ###
Num neurons in invariant 14
Neurion id and signature [(447, 0), (936, 0), (405, 0), (648, 1), (705, 0), (688, 0), (460, 1), (39, 1), (967, 0), (304, 1), (416, 0), (753, 1), (151, 1), (944, 1)]
Cluster size:  7
Is Consistent cluster:  True
Num misclassified 1

### Reference Image ###
Groundtruth 6
Prediction 8


### Cluster ###
Num neurons in invariant 12
Neurion id and signature [(447, 0), (936, 0), (405, 0), (648, 1), (705, 1), (976, 1), (183, 1), (861, 0), (688, 1), (893, 1), (408, 0), (774, 0)]
Cluster size:  2
Is Consistent cluster:  True
Num misclassified 1

### Reference Image ###
Groundtruth 6
Prediction 4


### Cluster ###
Num neurons in invariant 16
Neurion id and signature [(447, 1), (734, 0), (161, 1), (844, 1), (265, 1), (799, 1), (724, 0), (552, 1), (930, 1), (568, 0), (865, 1), (545, 0), (993, 0), (212, 0), (954, 0), (793, 0)]
Cluster size:  189
Is Consistent cluster:  True
Num misclassified 6

### Reference Image ###
Groundtruth 6
Prediction 5


...

### Cluster ###
Num neurons in invariant 14
Neurion id and signature [(447, 0), (936, 0), (405, 0), (648, 0), (831, 1), (45, 1), (728, 0), (230, 0), (46, 1), (721, 1), (151, 0), (256, 0), (871, 0), (626, 0)]
Cluster size:  3
Is Consistent cluster:  True
Num misclassified 2

### Reference Image ###
Groundtruth 6
Prediction 0


### Cluster ###
Num neurons in invariant 22
Neurion id and signature [(447, 0), (936, 1), (51, 1), (854, 0), (240, 1), (948, 0), (87, 1), (345, 0), (816, 1), (908, 0), (98, 0), (730, 0), (332, 0), (420, 1), (667, 0), (40, 0), (440, 0), (923, 1), (813, 0), (859, 0), (294, 0), (786, 1)]
Cluster size:  3667
Is Consistent cluster:  True
Num misclassified 22

### Reference Image ###
Groundtruth 6
Prediction 0


### Cluster ###
Num neurons in invariant 17
Neurion id and signature [(447, 0), (936, 0), (405, 0), (648, 1), (705, 1), (976, 0), (977, 0), (416, 0), (499, 0), (930, 1), (352, 1), (419, 1), (449, 0), (354, 1), (651, 0), (1023, 1), (248, 1)]
Cluster size:  1
Is Consistent cluster:  True
Num misclassified 1

### Reference Image ###
Groundtruth 6
Prediction 5


### Cluster ###
Num neurons in invariant 9
Neurion id and signature [(447, 0), (936, 1), (51, 1), (854, 1), (807, 0), (305, 0), (930, 1), (628, 0), (32, 0)]
Cluster size:  4
Is Consistent cluster:  True
Num misclassified 3

### Reference Image ###
Groundtruth 6
Prediction 5


### Cluster ###
Num neurons in invariant 13
Neurion id and signature [(447, 0), (936, 0), (405, 0), (648, 1), (705, 0), (688, 1), (320, 1), (977, 0), (478, 1), (598, 1), (261, 0), (18, 1), (571, 1)]
Cluster size:  3
Is Consistent cluster:  True
Num misclassified 1



### Visualizing the Decision Tree

In [0]:
dot_data = tree.export_graphviz(estimator, out_file=None) 
graph = graphviz.Source(dot_data)  
graph 